In [1]:
import os
from collections import Counter
from configparser import ConfigParser

import torch

from data_loader import load_sample_data
from evaluate import evaluate
from inference import generate_text
from tokenizer import tokenize_text
from train import create_batches, train_model
from transformer_model import TransformerModel

In [2]:
def get_hyperparameters():
    config = ConfigParser()
    config.read("config.ini")

    hyperparameters = config["Hyperparameters"]
    number_of_samples = int(hyperparameters["number_of_samples"])
    batch_size = int(hyperparameters["batch_size"])
    seq_length = int(hyperparameters["seq_length"])
    num_epochs = int(hyperparameters["num_epochs"])
    learning_rate = float(hyperparameters["learning_rate"])
    scheduler_patience = int(hyperparameters["scheduler_patience"])
    scheduler_factor = float(hyperparameters["scheduler_factor"])
    max_vocab_size = int(hyperparameters["max_vocab_size"])
    embedding_dim = int(hyperparameters["embedding_dim"])
    ff_hidden_dim = int(hyperparameters["ff_hidden_dim"])
    num_blocks = int(hyperparameters["num_blocks"])
    initial_text = hyperparameters["initial_text"]
    max_length = int(hyperparameters["max_length"])
    temperature = float(hyperparameters["temperature"])

    return (
        number_of_samples,
        batch_size,
        seq_length,
        num_epochs,
        learning_rate,
        scheduler_patience,
        scheduler_factor,
        max_vocab_size,
        embedding_dim,
        ff_hidden_dim,
        num_blocks,
        initial_text,
        max_length,
        temperature,
    )

In [3]:
# 1. Load hyperparameters
(
    number_of_samples,
    batch_size,
    seq_length,
    num_epochs,
    learning_rate,
    scheduler_patience,
    scheduler_factor,
    max_vocab_size,
    embedding_dim,
    ff_hidden_dim,
    num_blocks,
    initial_text,
    max_length,
    temperature,
) = get_hyperparameters()

In [4]:
# 2. Load and pre-process data
sample_text = load_sample_data(number_of_samples=number_of_samples)
vocab, word_to_idx, idx_to_word = tokenize_text(sample_text)
tokens = sample_text.split()

if len(tokens) > max_vocab_size:
    # Count the frequency of each word in your corpus
    word_freqs = Counter(tokens)

    # Get the most common words up to MAX_VOCAB_SIZE
    vocab = [word for word, freq in word_freqs.most_common(max_vocab_size - 1)]

    # Add the special <UNK> token to the vocabulary
    vocab.append("<UNK>")

    # Create word_to_idx dictionary
    word_to_idx = {word: idx for idx, word in enumerate(vocab)}

    # Replace all words not in the vocabulary with <UNK>
    tokens = [word if word in word_to_idx else "<UNK>" for word in tokens]

Skipping, found downloaded files in "./wikipedia-20230701" (use force=True to force download)


In [5]:
# 3. Create batches
input_batches, target_batches = create_batches(
    tokens, word_to_idx, batch_size=batch_size, seq_length=seq_length
)

In [6]:
# 4. Initialize or load model
model_path = "model.pth"
if os.path.exists(model_path):
    print("Loading existing model...")
    model = torch.load(model_path)
else:
    print("Initializing new model...")
    model = TransformerModel(
        vocab_size=max_vocab_size,
        embedding_dim=embedding_dim,
        ff_hidden_dim=ff_hidden_dim,
        num_blocks=num_blocks,
    )

Loading existing model...


In [14]:
# Reload hyperparameters before training (in case they've changed)
(
    number_of_samples,
    batch_size,
    seq_length,
    num_epochs,
    learning_rate,
    scheduler_patience,
    scheduler_factor,
    max_vocab_size,
    embedding_dim,
    ff_hidden_dim,
    num_blocks,
    initial_text,
    max_length,
    temperature,
) = get_hyperparameters()

# 5. Train the model
print("Training model...")
train_model(
    model,
    vocab,
    num_epochs,
    learning_rate,
    scheduler_patience,
    scheduler_factor,
    input_batches,
    target_batches,
)

# Save the trained model
print("Saving model...")
torch.save(model, model_path)

Training model...
Starting epoch 1/10:
Batch 1/4...2/4...3/4...4/4...Epoch 1/10, Loss: 4.128260552883148, Learning rate: 0.001
Starting epoch 2/10:
Batch 1/4...2/4...3/4...4/4...Epoch 2/10, Loss: 3.987725257873535, Learning rate: 0.001
Starting epoch 3/10:
Batch 1/4...2/4...3/4...4/4...Epoch 3/10, Loss: 3.5835296511650085, Learning rate: 0.001
Starting epoch 4/10:
Batch 1/4...2/4...3/4...4/4...Epoch 4/10, Loss: 3.2949923872947693, Learning rate: 0.001
Starting epoch 5/10:
Batch 1/4...2/4...3/4...4/4...Epoch 5/10, Loss: 3.002322554588318, Learning rate: 0.001
Starting epoch 6/10:
Batch 1/4...2/4...3/4...4/4...Epoch 6/10, Loss: 2.7230992317199707, Learning rate: 0.001
Starting epoch 7/10:
Batch 1/4...2/4...3/4...4/4...Epoch 7/10, Loss: 2.490257203578949, Learning rate: 0.001
Starting epoch 8/10:
Batch 1/4...2/4...3/4...4/4...Epoch 8/10, Loss: 2.271542191505432, Learning rate: 0.001
Starting epoch 9/10:
Batch 1/4...2/4...3/4...4/4...Epoch 9/10, Loss: 2.073909342288971, Learning rate: 0.00

KeyboardInterrupt: 

In [15]:
# 6. Evaluate the model
perplexity = evaluate(
    model,
    input_batches,
    target_batches,
    criterion=torch.nn.CrossEntropyLoss(),
    vocab_size=max_vocab_size,
)
print(f"Perplexity: {perplexity}")

Perplexity: 2.412450725474789


In [16]:
# 7. Generate text
generated_text = generate_text(
    model,
    idx_to_word,
    word_to_idx,
    initial_text=initial_text,
    max_length=max_length,
    temperature=temperature,
)
print(f"Generated Text: {generated_text}")

Generated Text: There de de de Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso Peso
